In [1]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.exceptions import UnexpectedResponse
from fastembed import TextEmbedding
from sentence_transformers import SentenceTransformer
import numpy as np
import requests

## Initialize a Qdrant Client

In [2]:
q_client = QdrantClient("http://localhost:6333")

## Read the course document

In [3]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

## Create a Collection

In [5]:
collection_name = 'vector-search'
SIZE = 512
try:
    q_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=SIZE,
            distance=models.Distance.COSINE,
        )
    )
except UnexpectedResponse as err:
    pass

## Create, Embed & Insert Points into the Collection

In [6]:
points = []
for course in documents_raw:
    for idx, doc in enumerate(course['documents']):
        point = models.PointStruct(
            id=idx,
            vector=models.Document(text=doc['text'], model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)       

In [ ]:
q_client.upsert(
    collection_name=collection_name,
    points=points
)

## Running a Similarity Search

In [ ]:
def search(query, collection_name, model_handle, limit=1):

    results = q_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
query = 'I just discovered the course. Can I join now?'
search(query, collection_name, model_handle)

## Q1) Embedding the query

In [ ]:
fast_model = TextEmbedding(model_name=model_handle)
embedding = next(fast_model.embed([query]))
print(f'Shape of the embedding {embedding.shape}')
print(f'Minimum value in embedding {embedding.min()}')

In [ ]:
results = q_client.search(
    collection_name=collection_name,
    query_vector=embedding,
    limit=5  # top 5 results
)

for res in results:
    print(res.payload['text'], 'Score:', res.score)

## Q2) Cosine similarity with another vector

In [ ]:
np.linalg.norm(embedding)

In [ ]:
vector = np.array(next(fast_model.embed([query])))

In [ ]:
vector.dot(vector)

In [ ]:
doc = 'Can I still join the course after the start date?'
other_vector = np.array(next(fast_model.embed([doc])))

In [ ]:
vector.dot(other_vector)

## Q3) Ranking by cosine

In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
doc_vectors = []
for doc in documents:
    text = doc['text']
    doc_vectors.append(next(fast_model.embed([text])))
doc_matrix = np.array(doc_vectors)

In [ ]:
len(doc_vectors)

In [ ]:
doc_vectors[0].shape

In [ ]:
[vector.dot(doc_vector) for doc_vector in doc_vectors]

In [ ]:
result = doc_matrix.dot(vector)

In [ ]:
result.argmax()

## Q4) Ranking by cosine, version two

In [ ]:
doc_vectors = []
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    doc_vectors.append(next(fast_model.embed([text])))
doc_matrix = np.array(doc_vectors)

In [ ]:
result = doc_matrix.dot(vector)

In [ ]:
result.argmax()

In [ ]:
result

In [ ]:
query

Yes, the results of ranking by consine in version 2 is improved as the question in the document is more closer to the query

## Q5) Selecting the embedding model

In [ ]:
models_list = TextEmbedding.list_supported_models()
model_dict = {model['model']: model['dim'] for model in models_list}
sorted(model_dict.items(), key=lambda k: (k[1], k[0]))[0]

## Q6) Indexing with qdrant (2 points)

In [ ]:
documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name == 'machine-learning-zoomcamp':
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)

In [ ]:
collection_name = 'vector-search-with-bge'
SIZE = 384
try:
    q_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=SIZE,
            distance=models.Distance.COSINE,
        )
    )
except UnexpectedResponse as err:
    pass

In [ ]:
points = []
model_handle = 'BAAI/bge-small-en'
for idx, doc in enumerate(documents):
    point = models.PointStruct(
        id=idx,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle),
        payload={
            "question": doc["question"],
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        }
    )
    points.append(point)   

In [ ]:
q_client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
search(query, collection_name, model_handle)